In [7]:
import sys
import os

src_path = os.path.abspath(os.path.join("..", "src"))
if src_path not in sys.path:
    sys.path.append(src_path)


from preprocess import generate_fake_landmark_data

generate_fake_landmark_data(num_samples=150, noise_level=0.5, label="mtbi")
generate_fake_landmark_data(num_samples=150, noise_level=0.1, label="control")

Saved simulated mtbi data as 'mtbi_sample_{i}.csv' in 'data/raw/split/'
Saved simulated control data as 'control_sample_{i}.csv' in 'data/raw/split/'


In [8]:
from features import (
    compute_inter_iris_distance,
    compute_average_diameter,
    normalise_distance,
    compute_time_series_std,
    flag_mTBI
)

In [9]:
import pandas as pd

df = pd.read_csv("data/raw/mtbi_sample.csv")
df.head()

,left_mid_x,left_mid_y,left_inner_x,left_inner_y,left_outer_x,left_outer_y,right_mid_x,right_mid_y,right_inner_x,right_inner_y,right_outer_x,right_outer_y
0,100.074164,99.752008,95.163949,99.752008,104.909682,99.752008,110.015772,99.869446,104.914795,99.869446,115.255582,99.869446
1,99.910565,99.931781,95.068107,99.931781,104.818702,99.931781,109.925951,100.111423,105.016521,100.111423,115.048391,100.111423
2,100.060091,100.137970,94.961230,100.137970,105.023174,100.137970,109.975893,100.100347,104.868243,100.100347,115.029552,100.100347
3,99.903180,100.128412,94.837560,100.128412,105.030663,100.128412,110.101146,99.988587,105.265123,99.988587,115.073087,99.988587
4,100.170700,99.906421,95.312364,99.906421,105.125633,99.906421,110.178668,99.755326,105.189726,99.755326,114.996366,99.755326


In [12]:
norm_distances = []

for i, row in df.iterrows():
    left_mid = (row["left_mid_x"], row["left_mid_y"])
    right_mid = (row["right_mid_x"], row["right_mid_y"])

    left_inner = (row["left_inner_x"], row["left_inner_y"])
    left_outer = (row["left_outer_x"], row["left_outer_y"])
    right_inner = (row["right_inner_x"], row["right_inner_y"])
    right_outer = (row["right_outer_x"], row["right_outer_y"])

    inter = compute_inter_iris_distance(left_mid, right_mid)
    avg_diameter = compute_average_diameter(left_inner, left_outer, right_inner, right_outer)
    norm = normalise_distance(inter, avg_diameter)

if norm is not None:
    norm_distances.append(norm)

# Remove any invalid values
norm_distances = [x for x in norm_distances if pd.notna(x)]

In [13]:
std_dev = compute_time_series_std(norm_distances)
is_mtbi = flag_mTBI(std_dev)

print(f"Standard deviation: {std_dev:.4f}")
print("You may possibly have a mTBI" if is_mtbi else "No mTBI detected")

Standard deviation: 0.0000
No mTBI detected
